# Enfield, Andrew - DATA 512, A2: Bias in Data

TBD UPDATE

The assignment is at https://wiki.communitydata.cc/HCDS_(Fall_2017)/Assignments#A2:_Bias_in_data.

TBD remove

This notebook pulls, prepares, and analyzes data about the amount of monthly English Wikipedia traffic from January 1, 2008 through September 30, 2017. For more information about the work and data, refer to the [README](Readme.md).

A few notes:
- Normally I'd prefer to keep the explanation and background that's in the README here in the notebook, so everything's in a single file, but I've split it up this time as that's what the assignment requested. I won't copy/paste because keeping duplicate content in sync is horrible.
- Real reproducibility needs tests for the code. A lot of my implementation below is in functions. I'd normally put these functions in at least one separate file that I import into this notebook, and I'd have tests in an additional file. For this assignment I'll just keep everything in this file, for simplicity, even though it means I can't test the code the way I normally would.

**TODO** I want a data section, where i note that there are issues, in one place. These include:

- articles that aren't actually a single politician, like 'History of monarchy in Canada	Canada	806849461
'
- follow on to above bullet that i deleted the Template: articles and why 
- something about Oliver's comments about multiple levels of hierarchy
- incomplete mapping of countries even after Oliver's and my follow on remapping
- 

# Prereqs

This code requires the libraries as described below.

In [2]:
# retrieve, load data
import requests
import json
import csv
import os

# prepare and analyze data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.axes_grid.anchored_artists import AnchoredText # for addtl annotations in charts
#from matplotlib.ticker import FuncFormatter # for custom axis labels
from IPython.core.pylabtools import figsize
import seaborn as sns # for formatting
%matplotlib inline 

In [3]:
sns.set_style("whitegrid")
figsize(14,7)

# Load and do basic data cleaning

TBD UPDATE

This section loads the data from the two APIs described in the README, producing five separate .json files, one for each API and access combination.

In [4]:
d_wikipedia = pd.read_csv('page_data.csv')
d_wikipedia.shape

(47197, 3)

In [5]:
d_wikipedia[:3]

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046


And, before we use the data to pull scores, we'll filter out the 'Template' entries.

**TODO** Note that while Oliver said he'd keep these entries, as they're evidence of coverage/activity, I'm going to drop them. I agree w/ Oliver, but I'm not sure that they're the same kind of coverage - are these apples to the actual per-politician page oranges? 

In [6]:
d_wikipedia.drop(d_wikipedia[d_wikipedia['page'].str.startswith('Template:')].index, inplace=True)
d_wikipedia.shape

(46701, 3)

In [7]:
d_wikipedia[:3]

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005


In [8]:
d_population = pd.read_csv('Population Mid-2015.csv', skiprows=2, thousands=',')
d_population.shape

(210, 6)

In [9]:
d_wikipedia.groupby(['country']).size().sort_values(ascending=False)[:10]

country
France            1681
Australia         1561
China             1133
United States     1092
Mexico            1077
Pakistan          1040
India              985
Russia             877
Spain              876
United Kingdom     863
dtype: int64

In [10]:
d_population[:3]

,Location,Location Type,TimeFrame,Data Type,Data,Footnotes
0,Afghanistan,Country,Mid-2015,Number,32247000,NaN
1,Albania,Country,Mid-2015,Number,2892000,NaN
2,Algeria,Country,Mid-2015,Number,39948000,NaN


## Pull article scores

Docs: https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model and https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context

Note that when I try the multiple rev ID API with a bunch of valid IDs and one that's a text string, then it gives me a 500 and no data at all. However when I try with a bunch of valid IDs and an ID that's not valid - like -1 - then I get valid/good data for the valid IDs and output like the following. This seems good: I'll go ahead and try just pulling batches of IDs w/o further error handling.

    "-1": {
        "wp10": {
          "error": {
            "message": "RevisionNotFound: Could not find revision ({revision}:-1)",
            "type": "RevisionNotFound"
          }
        }

In [11]:
user_agent = 'https://github.com/aenfield'

def get_multiple_full_ores_score_json(rev_ids):
    """TBD referring to ..., with rev_ids as a list"""
    endpoint = 'https://ores.wikimedia.org/v3/scores/enwiki?models=wp10&revids={rev_ids_delimited}'

    rev_ids_delimited = '|'.join([str(i) for i in rev_ids])

    params = { 'rev_ids_delimited' : rev_ids_delimited }

    api_call = requests.get(endpoint.format(**params), headers = {'User-Agent':'{}'.format(user_agent)})
    return api_call.json()
    
def get_ores_prediction_from_score_json(score_json, rev_id):
    """Return the most likely article type, per ORES. Assumes a JSON dict from Ores. """
    #print(rev_id)
    try:
        return score_json['enwiki']['scores'][str(rev_id)]['wp10']['score']['prediction']
    except KeyError as err:
        return f"KeyError: {err}."
            
def chunker(seq, size):
    """Get a generator that returns chunks of size 'size' of the sequence in 'seq'.
    
    From: https://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks
    """
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_article_scores_data(rev_ids, force=False, verbose=False):
    """TBD update Download and save results from the specified API to a local file, by default only if local file doesn't exist.
    TBD call with d_wikipedia['last_edit'].values for 'rev_ids'
    
    apiname - 'pagecounts' or 'pageviews'
    params - a dict containing 'access', 'start', and 'end' keys; use get_param_dict_from_params
    user_agent - an identifier for the user making the request; can be a GitHub user URL or general email address
    force - download data and overwrite local file, even if file already exists; default is False
    verbose - print diagnostic data; default is false
    """
    
    filename = 'article_scores.csv'

    if (not os.path.exists(filename)) | (force == True):
        # download and save the data locally
        if verbose: print("Local file doesn't exist or download was forced. Downloading...")
        with open(filename, 'w') as output_file:
            writer = csv.writer(output_file, delimiter=',')
            writer.writerow(['RevisionId','Score'])

            progress_frequency = 25
            count_of_rev_ids_in_chunk = 140
            rev_ids_in_chunks = [x for x in chunker(rev_ids, count_of_rev_ids_in_chunk)]

            #rev_ids_in_chunks = rev_ids_in_chunks[300:]
            
            for chunk_index, chunk_of_rev_ids in enumerate(rev_ids_in_chunks):
                if (chunk_index % progress_frequency == 0): print(f"Retrieving chunk with index {chunk_index}.")

                scores_json = get_multiple_full_ores_score_json(chunk_of_rev_ids)
                #print(scores_json)
                for rev_id in chunk_of_rev_ids:
                    writer.writerow([rev_id, get_ores_prediction_from_score_json(scores_json, rev_id)])

            if verbose: print(f"Retrieved {chunk_index + 1} chunks and saved to {filename}. Done.")        
    else:
        if verbose: print("Local file exists already.")
                  
    # now open and return dataframe
    return pd.read_csv(filename, index_col='RevisionId')

In [12]:
d_scores = get_article_scores_data(d_wikipedia['rev_id'].values)
d_scores.shape

(46701, 1)

Some articles may have been deleted since the page_data list was created. How many of these occurred? And then we'll filter them out, since we can't get a score for something that doesn't exist.

In [13]:
len(d_scores[d_scores['Score'].str.startswith('KeyError')])

2

In [14]:
d_scores[d_scores['Score'].str.startswith('KeyError')]

,Score
RevisionId,
807367030,KeyError: 'score'.
807367166,KeyError: 'score'.


In [15]:
d_scores.drop(d_scores[d_scores['Score'].str.startswith('KeyError')].index, inplace=True)
d_scores.shape

(46699, 1)

In [16]:
len(d_scores[d_scores['Score'].str.startswith('KeyError')])

0

And now join to pull in the score data.

If there are rows with no score data, the page data will have nulls for the new score column. I'll filter these rows out below when I clean up rows with incomplete data.

In [17]:
d_wikipedia_with_scores = pd.merge(left=d_wikipedia, right=d_scores, left_on='rev_id', right_index=True, how='left')
d_wikipedia_with_scores.shape

(46701, 4)

In [18]:
d_wikipedia_with_scores[:3]

,page,country,rev_id,Score
1,Bir I of Kanem,Chad,355319463,Stub
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
12,Yos Por,Cambodia,393822005,Stub


In [19]:
d_wikipedia_with_scores['Score'].value_counts(dropna=False)

Stub     23707
Start    15341
C         5764
GA         872
B          735
FA         280
NaN          2
Name: Score, dtype: int64

In [20]:
d_wikipedia.shape

(46701, 3)

In [21]:
len(d_wikipedia_with_scores) - len(d_wikipedia)

0

In [22]:
d_scores.shape

(46699, 1)

In [23]:
d_wikipedia_with_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46701 entries, 1 to 47196
Data columns (total 4 columns):
page       46701 non-null object
country    46701 non-null object
rev_id     46701 non-null int64
Score      46699 non-null object
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [24]:
# d_wikipedia_with_scores.to_excel('eyeball.xlsx')

In [25]:
len(d_wikipedia_with_scores['rev_id'].unique())

46701

In [26]:
len(d_wikipedia['rev_id'].unique())

46701

In [27]:
len(d_scores.index.unique())

46699

Originally the page_data data set had duplicates. It looks like that's no longer the case, but I'll check for them here to confirm.

In [28]:
dupes = d_wikipedia_with_scores[d_wikipedia_with_scores.duplicated(subset='rev_id', keep=False)].sort_values(['rev_id','page'])
dupes

,page,country,rev_id,Score


Join to pull in the population data. For now we'll do an outer join so we can see the country values that don't exist on _both_ sides of the join. This isn't needed for the assignment because it says to just remove all rows that don't have matching data. But, I'm curious and I also want to at least see if there are places where I could do any further matching to expand/improve the data.

In [29]:
d_all = pd.merge(left=d_wikipedia_with_scores, right=d_population[['Location','Data']],
                 how='outer', left_on='country', right_on='Location')
d_all.shape

(46724, 6)

In [30]:
d_all[:3]

,page,country,rev_id,Score,Location,Data
0,Bir I of Kanem,Chad,355319463.0,Stub,Chad,13707000.0
1,Abdullah II of Kanem,Chad,498683267.0,Stub,Chad,13707000.0
2,Salmama II of Kanem,Chad,565745353.0,Stub,Chad,13707000.0


Now show the countries that exist on each side but not on the other.

First, countries that exist in the Wikipedia data but not in the population data, w/ the number of rows - here, the number of pages - for each

In [31]:
d_all[d_all['Location'].isnull()]['country'].value_counts(dropna=False)

Hondura                             187
Salvadoran                          116
South Korean                         96
Cape Colony                          81
Samoan                               76
Rhodesian                            75
Faroese                              74
Ivorian                              73
Cook Island                          67
Jersey                               61
Saint Lucian                         47
Pitcairn Islands                     43
Chechen                              38
East Timorese                        36
Saint Kitts and Nevis                30
Montserratian                        27
Guernsey                             25
Omani                                24
Carniolan                            22
Niuean                               22
Palauan                              21
Saint Vincent and the Grenadines     21
South Ossetian                       18
Tokelauan                            17
Abkhazia                             16


And the countries that exist in the population data but not in the Wikipedia data.

In [32]:
d_all[d_all['country'].isnull()]['Location'].value_counts(dropna=False)

Hong Kong, SAR                  1
New Caledonia                   1
St. Vincent & the Grenadines    1
Samoa                           1
Macao, SAR                      1
St. Kitts-Nevis                 1
El Salvador                     1
Puerto Rico                     1
Guam                            1
Palau                           1
Channel Islands                 1
Western Sahara                  1
Mayotte                         1
French Polynesia                1
Cote d'Ivoire                   1
Reunion                         1
Oman                            1
St. Lucia                       1
Honduras                        1
Brunei                          1
Georgia                         1
Timor-Leste                     1
Curacao                         1
Name: Location, dtype: int64

In [33]:
#d_population.to_excel('population.xlsx')
#d_wikipedia_with_scores['country'].value_counts(dropna=False).reset_index().to_excel('page countries.xlsx')

Based on eyeballing of the most common Wikipedia countries for which we have no population data - I included all countries with 70 or more pages - I came up with the following mapping that I'll use to update the country field in the Wikipedia data, so we _will_ have population data. This is probably clearly better for some countries that map directly - for example updating 'South Korean' to 'Korea, South'. It's fuzzier for at least some others, including for example Wikipedia countries that aren't around anymore in the way they were when the leader was in power, like Cape Colony and Rhodesia. Good enough for now though.

In addition, there are clearly others that I could also map - for example, 'Saint Vincent and the Grenadines' in the Wikipedia data could be changed to 'St. Vincent & the Grenadines', but all of the ones I didn't manually map have fewer than 70 pages.

In [34]:
wikipedia_country_to_population_map = {'Burkinabé':'Burkina Faso',
                                       'Ivorian':"Cote d'Ivoire",
                                       'Faroese':'Denmark',
                                       'Salvadoran':'El Salvador',
                                       'Hondura':'Honduras',
                                       'South Korean':'Korea, South',
                                       'Samoan':'Samoa',
                                       'Cape Colony':'United Kingdom',
                                       'Rhodesian':'Zimbabwe'}

In [35]:
d_wikipedia_with_scores['country'].update(d_wikipedia_with_scores['country'].map(wikipedia_country_to_population_map))

And then we need to re-merge the population data.

**TODO** This is duplication. DRY.

In [36]:
d_all = pd.merge(left=d_wikipedia_with_scores, right=d_population[['Location','Data']],
                 how='outer', left_on='country', right_on='Location')
d_all.shape

(46720, 6)

In [37]:
d_all[d_all['Location'].isnull()]['country'].value_counts(dropna=False)

Cook Island                         67
Jersey                              61
Saint Lucian                        47
Pitcairn Islands                    43
Chechen                             38
East Timorese                       36
Saint Kitts and Nevis               30
Montserratian                       27
Guernsey                            25
Omani                               24
Niuean                              22
Carniolan                           22
Saint Vincent and the Grenadines    21
Palauan                             21
South Ossetian                      18
Tokelauan                           17
Abkhazia                            16
South African Republic              15
Greenlandic                         13
Ossetian                             9
Incan                                7
Dagestani                            7
Somaliland                           4
Rojava                               2
Name: country, dtype: int64

In [38]:
d_all[d_all['country'].isnull()]['Location'].value_counts(dropna=False)

Hong Kong, SAR                  1
Channel Islands                 1
St. Vincent & the Grenadines    1
Timor-Leste                     1
Macao, SAR                      1
St. Kitts-Nevis                 1
Puerto Rico                     1
Guam                            1
Palau                           1
New Caledonia                   1
Mayotte                         1
Western Sahara                  1
French Polynesia                1
Reunion                         1
Oman                            1
St. Lucia                       1
Brunei                          1
Georgia                         1
Curacao                         1
Name: Location, dtype: int64

In [39]:
d_all[:3]

,page,country,rev_id,Score,Location,Data
0,Bir I of Kanem,Chad,355319463.0,Stub,Chad,13707000.0
1,Abdullah II of Kanem,Chad,498683267.0,Stub,Chad,13707000.0
2,Salmama II of Kanem,Chad,565745353.0,Stub,Chad,13707000.0


And now that we've added in the additional remappings here, rows that still have a NaN 'Location' value are rows for which we have no population data and, per the assignment, we can get rid of them.

In [40]:
d_all.shape

(46720, 6)

In [41]:
sum(d_all['Location'].isnull())

592

In [42]:
d_all.dropna(subset=['Location'], inplace=True)
d_all.shape

(46128, 6)

Since we did an outer join above so we could understand data without matches on both sides of the join, we also need to drop the rows with location/population data but with no matching articles. These rows have nulls for the page data fields - for page, country, rev_id, and Score.

In [43]:
d_all.dropna(subset=['page', 'country', 'rev_id', 'Score'], how='all', inplace=True)
d_all.shape

(46109, 6)

And we'll also get rid of any rows where we don't have score data - these can come from, for example as noted above, cases where the actual page has been deleted on Wikipedia.

In [44]:
d_all[d_all['Score'].isnull()]

,page,country,rev_id,Score,Location,Data
9352,Jalal Movaghar,Iran,807367030.0,NaN,Iran,78483446.0
9353,Mohsen Movaghar,Iran,807367166.0,NaN,Iran,78483446.0


In [45]:
d_all.dropna(subset=['Score'], inplace=True)
d_all.shape

(46107, 6)

Clean up columns to match assignment instructions: rename and drop extra join column.

In [46]:
del d_all['Location']
d_all.rename(columns={'page':'article_name',
                      'rev_id':'revision_id',
                      'Score':'article_quality',
                      'Data':'population'}, inplace=True)
d_all[:3]

,article_name,country,revision_id,article_quality,population
0,Bir I of Kanem,Chad,355319463.0,Stub,13707000.0
1,Abdullah II of Kanem,Chad,498683267.0,Stub,13707000.0
2,Salmama II of Kanem,Chad,565745353.0,Stub,13707000.0


# Analysis

In [150]:
articles_per_person = d_all.groupby(['country']).apply(lambda g: len(g) / g.iloc[0]['population'])
articles_per_person[:3]

country
Afghanistan    0.000010
Albania        0.000158
Algeria        0.000003
dtype: float64

In [156]:
articles_per_person['United States']

3.3993892779252636e-06

In [157]:
articles_per_person['Norway']

0.00012628958316929483

In [158]:
articles_per_person.sort_values(ascending=False)

country
Nauru                             4.788214e-03
Tuvalu                            4.576271e-03
San Marino                        2.454545e-03
Monaco                            1.050200e-03
Liechtenstein                     7.452755e-04
Marshall Islands                  6.727273e-04
Iceland                           6.105892e-04
Tonga                             6.098742e-04
Andorra                           4.358974e-04
Samoa                             3.913290e-04
Federated States of Micronesia    3.495146e-04
Grenada                           3.243243e-04
Luxembourg                        3.127185e-04
Antigua and Barbuda               2.666667e-04
Kiribati                          2.645503e-04
Maldives                          2.392303e-04
Malta                             2.387099e-04
Fiji                              2.283737e-04
Seychelles                        2.262127e-04
Vanuatu                           2.162162e-04
Dominica                          1.764706e-04
New Z

In [159]:
def is_high_quality(score):
    if (score == 'FA') | (score == 'GA'):
        return True
    else:
        return False

In [160]:
d_all['article_quality'].value_counts(dropna=False)

Stub     23456
Start    15103
C         5685
GA         862
B          722
FA         279
Name: article_quality, dtype: int64

In [161]:
(d_all['article_quality'].apply(is_high_quality)).value_counts(dropna=False)

False    44966
True      1141
Name: article_quality, dtype: int64

In [162]:
# define this instead of using a lambda, like above, so we can keep the lines around 80 chars wide
# a lambda would be fine as the actual function is still pretty short/a single liner
def get_high_quality_article_proportion(articles):
    return sum(articles['article_quality'].apply(is_high_quality)) / len(articles)

high_quality_articles_per_all_articles = d_all.groupby(['country']).apply(get_high_quality_article_proportion)
high_quality_articles_per_all_articles[:3]

country
Afghanistan    0.059006
Albania        0.010941
Algeria        0.025862
dtype: float64

In [163]:
high_quality_articles_per_all_articles.sort_values(ascending=False)

country
Korea, North                      0.250000
Romania                           0.131195
Saudi Arabia                      0.127119
Central African Republic          0.121212
Guinea-Bissau                     0.100000
Qatar                             0.100000
Vietnam                           0.096257
Bhutan                            0.090909
Mauritania                        0.083333
Ireland                           0.082011
United States                     0.078755
Singapore                         0.072464
Guatemala                         0.072289
Uzbekistan                        0.071429
Palestinian Territory             0.067039
Benin                             0.065934
Syria                             0.062016
Gabon                             0.061224
United Kingdom                    0.060381
Afghanistan                       0.059006
Ukraine                           0.056667
Vanuatu                           0.050000
Congo, Dem. Rep. of               0.049296
Jam

If we filter out countries with 'smaller' populations, what do we see for the same lists? First we'll add in the population data and then we can use that to filter the lists.

In [172]:
articles_per_person = pd.concat([articles_per_person, 
           d_population[['Location','Data']].set_index('Location')['Data']],
           axis=1).rename(columns={0:'proportion','Data':'population'})
articles_per_person[:3]

,proportion,population
Afghanistan,0.000010,32247000
Albania,0.000158,2892000
Algeria,0.000003,39948000


In [173]:
high_quality_articles_per_all_articles = pd.concat([high_quality_articles_per_all_articles, 
           d_population[['Location','Data']].set_index('Location')['Data']],
           axis=1).rename(columns={0:'proportion','Data':'population'})
high_quality_articles_per_all_articles[:3]

,proportion,population
Afghanistan,0.059006,32247000
Albania,0.010941,2892000
Algeria,0.025862,39948000


And now we can look at the lists again, with a threshold population.

In [174]:
pop_thresh = 50000000

In [175]:
articles_per_person[articles_per_person['population'] > pop_thresh].sort_values(by='proportion', ascending=False)['proportion']

France                 2.612410e-05
United Kingdom         1.450255e-05
Italy                  1.319101e-05
Iran                   1.046080e-05
Germany                8.516985e-06
Mexico                 8.479180e-06
Tanzania               7.745119e-06
Korea, South           7.394427e-06
South Africa           6.867608e-06
Russia                 6.077532e-06
Pakistan               5.224889e-06
Philippines            4.943413e-06
Myanmar                4.544844e-06
Turkey                 4.449274e-06
Nigeria                3.734064e-06
Japan                  3.444557e-06
United States          3.399389e-06
Brazil                 2.699010e-06
Egypt                  2.660709e-06
Vietnam                2.038945e-06
Bangladesh             2.001110e-06
Congo, Dem. Rep. of    1.936182e-06
Thailand               1.719869e-06
Ethiopia               1.029058e-06
China                  8.258499e-07
Indonesia              8.250503e-07
India                  7.495638e-07
Name: proportion, dtype: flo

In [176]:
high_quality_articles_per_all_articles[high_quality_articles_per_all_articles['population'] > pop_thresh].sort_values(by='proportion', ascending=False)['proportion']

Vietnam                0.096257
United States          0.078755
United Kingdom         0.060381
Congo, Dem. Rep. of    0.049296
Egypt                  0.046414
Philippines            0.043222
Indonesia              0.042654
Russia                 0.039909
Myanmar                0.037975
China                  0.037070
South Africa           0.031746
Germany                0.027496
Thailand               0.026786
Iran                   0.020706
Ethiopia               0.019802
Bangladesh             0.018692
Korea, South           0.018667
France                 0.017847
Japan                  0.016018
India                  0.015228
Pakistan               0.013462
Turkey                 0.011494
Italy                  0.009709
Brazil                 0.009058
Mexico                 0.006500
Nigeria                0.005891
Tanzania               0.002469
Name: proportion, dtype: float64

In [ ]:
d_all.to_csv('en-wikipedia-politician-scores.csv', index=False)